In [1]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=689007 sha256=3efa6631bcd70463c03109233c81dc1cb483c253e966eaf2fc68ba778a442dc3
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from google.colab import drive

In [3]:
drive.mount('/content/drive')
device = torch.device("cuda:0")

Mounted at /content/drive


In [4]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))

In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8, # keywords 라벨 개수
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [7]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [8]:
%cd './drive/MyDrive/Colab Notebooks/ufriends-chatbot/'

/content/drive/MyDrive/Colab Notebooks/ufriends-chatbot


In [9]:
!pwd

/content/drive/MyDrive/Colab Notebooks/ufriends-chatbot


In [10]:
train_df = pd.read_csv('Dataset/train_df_preprocess.csv')
validation_df = pd.read_csv('Dataset/validation_df_preprocess.csv')

In [11]:
train_df

,keywords,major_emotions,user1,chatbot1
0,2.0,5.0,아내가 드디어 출산하게 되어서 정말 신이 나,아내분이 출산을 하시는군요 정말 축하드려요
1,3.0,2.0,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야,약 종류가 많아 번거로우시겠어요
2,5.0,4.0,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요
3,4.0,5.0,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해,재취업 후 첫 월급이라 정말 기쁘시겠어요
4,4.0,5.0,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어,기분 좋으시겠어요 앞으로는 어떻게 하실 생각이신가요
...,...,...,...,...
38962,3.0,5.0,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어,검진 결과가 좋게 나와서 안도하겠네요
38963,7.0,1.0,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래 너무 슬퍼,어머니의 응원이 사라져서 슬프시겠어요 조금 더 자세히 말해주시겠어요
38964,4.0,5.0,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야,운동 시설을 쓸 수 있어서 기분이 좋으시겠어요
38965,0.0,2.0,친구 관계가 너무 힘들어 베푸는 만큼 돌아오지 않는 것 같아,많이 서운하셨겠어요 친구들에게 서운함을 느꼈을 때 해결하는 방법이 있나요


In [13]:
train_set = train_df.loc[:, ['keywords', 'user1']]
validation_set = validation_df.loc[:, ['keywords', 'user1']]

In [14]:
train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)

In [15]:
train_set_data = [[i, str(j)] for i, j in zip(train_set['user1'], train_set['keywords'])]
validation_set_data = [[i, str(j)] for i, j in zip(validation_set['user1'], validation_set['keywords'])]

train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=4)

train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=2)

In [16]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# epoch을 5로 설정
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    # train set
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # test set
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-16-daa5795bd9ea>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/488 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.096160411834717 train acc 0.1875
epoch 1 batch id 201 loss 1.3551757335662842 train acc 0.4201648009950249
epoch 1 batch id 401 loss 0.8990326523780823 train acc 0.5590321072319202
epoch 1 train acc 0.5867059426229508


<ipython-input-16-daa5795bd9ea>:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/122 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7536475409836065


  0%|          | 0/488 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.502150297164917 train acc 0.875
epoch 5 batch id 201 loss 0.5105624198913574 train acc 0.8781094527363185
epoch 5 batch id 401 loss 0.39249032735824585 train acc 0.8855205735660848
epoch 5 train acc 0.8852138831967213


  0%|          | 0/122 [00:00<?, ?it/s]

epoch 5 test acc 0.7584426229508197


In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/ufriends-chatbot


In [26]:
torch.save(model, f'Model/keywords-SentimentAnalysisKOBert.pt')
torch.save(model.state_dict(), f'Model/keywords-SentimentAnalysisKOBert_StateDict.pt')

In [ ]:
# 대인관계, 대인관계(부부, 자녀) - 0 
# 진로, 취업, 직장, 업무 스트레스 - 1
# 연애, 결혼, 출산 - 2
# 건강, 죽음 - 3
# 재정, 은퇴, 노후준비 - 4
# 학업 및 진로 - 5
# 학교폭력, 따돌림 - 6
# 가족관계 - 7

In [21]:
sentence = '친구랑 싸워서 속상해'
predict(sentence)

0

In [18]:
sentence = '회사에서 해고 당했어'
predict(sentence)

1

In [19]:
sentence = '아이를 키우려면 돈이 많이 들어'
predict(sentence)

2

In [24]:
sentence = '비타민 챙겨 먹어야지'
predict(sentence)

3

In [25]:
sentence = '자식들과 여행하며 살고싶어'
predict(sentence)

4

In [20]:
sentence = '시험에서 떨어지면 어떡하지?'
predict(sentence)

5

In [23]:
sentence = '친구들을 괴롭히는 것은 나빠'
predict(sentence)

6

In [22]:
sentence = '부모님과 싸웠어'
predict(sentence)

7